In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
from gensim.corpora.dictionary import Dictionary
from gensim.models.phrases import Phrases, Phraser
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import yake
from langdetect import detect, detect_langs, DetectorFactory
import fasttext
import re

In [2]:
# read csv
authors = pd.read_csv('top_20_authors.csv')
publications = pd.read_csv('publications-top_20_authors.csv', sep=',')
# authors = pd.read_csv('some_authors.csv')
# publications = pd.read_csv('publications-some_authors.csv', sep=',')

In [47]:
# make a dictionary containing the combined abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))
texts = {author_id: [] for author_id in authors["id"]}

# load fastText model
model = fasttext.load_model('lid.176.bin')

for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        # predict the language
        predictions = model.predict(abstract)
        language = predictions[0][0].replace('__label__', '')
        # keep only texts written in English
        if language == 'en':
            texts[author_id].append(abstract)

In [ ]:
text = authors_texts[802]
doc = nlp(text)

print(len(doc.ents))
for token in doc:
    if "politehnica" in token.text.lower() or "polytechnic" in token.text.lower() or "faculty" in token.text.lower() or "univeristy" in token.text.lower():
        print(token.text, token.ent_type_)

In [37]:
stop_words = ['abstract', 'amount', 'approach', 'article', 'available', 'base', 'based', 'benefit',
              'bucharest',
              'case', 'category', 'condition', 'conference', 'context', 'copyright', 'datum', 'demonstrate',
              'demonstrates', 'demonstrated',
              'different', 'difficult', 'experiment', 'experimental', 'faculty', 'helpful', 'high',
              'ieee', 'importance', 'important', 'inconvenience', 'interest', 'interested', 'interests', 'jat',
              'jats', 'laboratory',
              'main', 'new', 'obtain', 'obtained', 'obtains', 'old', 'order', 'organization', 'paper', 'people',
              'policy',
              'politehnica', 'polytechnic',
              'present', 'presents', 'presented', 'privacy', 'professor', 'propose', 'proposes', 'proposed',
              'quality', 'range', 'ranges', 'real',
              'recent', 'research', 'researcher', 'result', 'scale', 'show', 'shows', 'showed', 'student', 'study',
              'subject', 'studies', 'studied', 'task',
              'teacher', 'term', 'text', 'title', 'type', 'unavailable', 'university', 'useful',
              'workshop']

EXTRACT KEYWORDS/KEYPHRASES WITH YAKE

In [25]:
nlp = spacy.load('en_core_web_lg')

In [49]:
abstract_list = texts[1047]

In [51]:
# remove abstracts that contain mostly person and organization names
clean_abstract_list = []

for abstract in abstract_list:
    nlp.max_length = len(abstract) + 1000
    doc = nlp(abstract)
    person_orgs_count = 0
    other_words_count = 0

    for token in doc:
        if token.ent_type_ == 'PERSON' or token.ent_type_ == 'ORG':
            person_orgs_count += 1
        elif token.is_alpha:
            other_words_count += 1

    if person_orgs_count < other_words_count:
        clean_abstract_list.append(abstract)

In [52]:
# concatenate the abstracts into a single string
text = '\n'.join(clean_abstract_list)

In [53]:
# remove some named entities
nlp.max_length = len(text) + 1000
doc = nlp(text)
remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']

transformed_text = ' '.join([token.text for token in doc if token.ent_type_ not in remove_entities])


In [54]:
# set parameters for yake keyword extractor
max_ngram = 3
deduplication_threshold = 0.5
keywords_nr = 15
windowsSize = 1

kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold,
                                     top=keywords_nr, windowsSize=windowsSize)
# add custom stop words to the default set from yake
kw_extractor.stopword_set.update(set(stop_words))

# extract keywords
keywords = kw_extractor.extract_keywords(transformed_text)
for kw, _ in keywords:
    print(kw)

mobile working machine
finite element method
grape seed oil
agricultural machines working
soil compaction soil
soil tillage machines
compaction soil compaction
independent agricultural machines
soil modelling equipment
Gasification process TLUD
hydraulic PET waste
PET waste press
Active suspension systems
protective agricultural equipment
machines working parts


In [42]:
from termcolor import colored

# Funcție pentru evidențierea cuvintelor cheie
def highlight_keywords(text, keywords):
    for kw, _ in keywords:
        text = text.replace(kw, colored(kw, 'red', attrs=['bold']))
    return text

# Evidențierea cuvintelor cheie în text
highlighted_text = highlight_keywords(text, keywords)
print(highlighted_text)


Environmental Engineering and Management Journal. Article Tools Print this article. How to cite item. Journal Content Search. Search Scope All. Browse …
Monodisperse coreshell hybrid nanoparticles based on cobalt ferrite (CoFe2O4) particles coated with polyvinylpyrrolidone (PVP) or polyethylene glycol (PEG) biopolymers were obtained employing a two-step procedure: the CoFe2O4 of 21nm mean particle size were first synthesized by coprecipitation method assisted by PVP soft template and then were coated by PVP or PEG biopolymers. The effect of the thermal treatment upon the phase evolution of the obtained precursor from the coprecipitation step was monitored by X-ray diffraction (XRD) and Fourier transform infrared spectroscopy (FTIR) analyses. The FTIR spectra also indicated the interaction between the cobalt ferrite particles and the two polymers were used as biocompatible coatings. Transmission electron microscopy (TEM) and Selected Area Electron Diffraction (SAED) analyses revealed t

EXTRACT TOPICS WITH GENSIM LDA

In [56]:
abstract_list = texts[562]

In [57]:
# remove abstracts that contain mostly person and organization names
clean_abstract_list = []

for abstract in abstract_list:
    nlp.max_length = len(abstract) + 1000
    doc = nlp(abstract)
    person_orgs_count = 0
    other_words_count = 0

    for token in doc:
        if token.ent_type_ == 'PERSON' or token.ent_type_ == 'ORG':
            person_orgs_count += 1
        elif token.is_alpha:
            other_words_count += 1

    if person_orgs_count < other_words_count:
        clean_abstract_list.append(abstract)

In [59]:
# keep only adjectives and nouns
remove_pos = ['ADV', 'PRON', 'PART', 'DET', 'SPACE', 'NUM', 'SYM', 'ADP', 'VERB', 'CCONJ']
remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']

tokens = []
for abstract in clean_abstract_list:
    nlp.max_length = len(abstract) + 1000
    doc = nlp(abstract)
    t = [token.lemma_.lower() for token in doc if token.is_alpha and token.ent_type_ not in remove_entities
                             and token.lemma_.lower() not in stop_words and token.pos_ not in remove_pos and not token.is_stop]
    tokens.append(t)

# add bigrams to the token list
bigram = Phrases(tokens, min_count=2, delimiter=' ', threshold=1)
tokens = [bigram[text] for text in tokens]
trigram = Phrases(tokens, min_count=2, delimiter=' ', threshold=1)
tokens = [trigram[text] for text in tokens]

In [60]:
# remove single words
ngrams = [[token for token in text if len(token.split(" ")) > 1] for text in tokens]

In [61]:
word_count = 0
for text in ngrams:
    word_count += len(text)

if word_count > 100:
    tokens = ngrams

In [62]:
# create dictionary with gensim
dictionary = Dictionary(tokens)
# create corpus
corpus = [dictionary.doc2bow(text) for text in tokens]

In [63]:
# apply LDA
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=300, num_topics=1, workers=4, passes=50)
topics = lda_model.print_topics(-1)

for idx, topic in topics:
    print(topic)

0.016*"scheduling algorithm" + 0.015*"cloud computing" + 0.010*"big data" + 0.009*"resource management" + 0.006*"cloud system" + 0.006*"smart city" + 0.006*"time series" + 0.006*"large system" + 0.006*"genetic algorithm" + 0.006*"web service"


In [22]:
# try ner labeling with bert
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

text = authors_texts[841]
ner_results = nlp(text)

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
for result in ner_results:
    print(f"{result['word']} {result['entity']}")

print(text)

Fe B-MISC

Ferrofluids consist of circa 100Angstrom diameter particles of a magnetic solid (usually magnetite, Fe3O4) colloidally suspended in a carrier fluid. Typical carrier fluids include hydrocarbons, water, fluorocarbons, esters, diesters, organometallics, polyphenyl ethers and silicones. Surfactants such as oleic acid and other small molecules as well as a few polymers have been used to coat the surface of microcrystalline Fe3O4 to help prevent particle flocculation. It is of considerable interest, thus, to develop better synthetic approaches for ferrofluids, particularly since their unusual magnetic properties are finding applications in loudspeakers, zero-leakage seals, damping liquids, non-wearing electrical switches, artificial muscles, magnetic inks, environmental protection and energy converters. The synthesis and the systematic study of the properties of magnetic fluids (magnetic dispersions) were started a few years ago. In …
The liquid membranes based on ferrofluid have 